In [ ]:
from sklearn.model_selection import train_test_split

import columns
import paths


def split_train_test(X, y, test_size=0.2, random_state=42, stratify=None):
    return train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=stratify
    )

# Fire occurrence model (classification)


In [ ]:
from catboost import CatBoostClassifier
import pandas as pd

df = pd.read_csv(paths.inmet_inpe_linked_file)

Xc = df.drop(columns=[columns.fire_intensity_column, columns.fire_occurrence_column])
yc = df[columns.fire_occurrence_column]

Xc_train, Xc_test, yc_train, yc_test = split_train_test(Xc, yc, stratify=yc)

pos = yc.sum()
neg = len(yc) - pos
scale = neg / pos

model_cls = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    learning_rate=0.05,
    depth=8,
    iterations=300,
    scale_pos_weight=scale,
    verbose=False,
    train_dir=paths.classification_info_dir
)

model_cls.fit(Xc_train, yc_train, eval_set=(Xc_test, yc_test))
model_cls.save_model(paths.classification_cbm_file)

# Fire intensity model (regression)

In [ ]:
from catboost import CatBoostRegressor

df_fire = df[df.fire_occurred == 1]

Xr = df_fire.drop(columns=[columns.fire_intensity_column, columns.fire_occurrence_column])
yr = df_fire[columns.fire_intensity_column]

Xr_train, Xr_test, yr_train, yr_test = split_train_test(Xr, yr)

model_reg = CatBoostRegressor(
    loss_function='RMSE',
    learning_rate=0.05,
    depth=8,
    iterations=300,
    verbose=False,
    train_dir=paths.regression_info_dir
)

model_reg.fit(Xr_train, yr_train, eval_set=(Xr_test, yr_test))
model_reg.save_model(paths.regression_cbm_file)